In [ ]:
import numpy as np
from PIL import Image

Manual RGB to YCbCr Conversion


In [ ]:
def rgb_to_ycbcr(rgb):
    ycbcr = np.zeros_like(rgb, dtype=float)
    for i in range(rgb.shape[0]):
        for j in range(rgb.shape[1]):
            r, g, b = rgb[i, j]
            y = 0.299 * r + 0.587 * g + 0.114 * b
            cb = -0.1687 * r - 0.3313 * g + 0.5 * b + 128
            cr = 0.5 * r - 0.4187 * g - 0.0813 * b + 128
            ycbcr[i, j] = [y, cb, cr]
    return ycbcr

Subsampling 4:2:0


In [ ]:
def subsample(ycbcr):
    y = ycbcr[:, :, 0]
    cb = ycbcr[::2, ::2, 1]
    cr = ycbcr[::2, ::2, 2]
    return y, cb, cr

2D DCT on 8x8 blocks


In [ ]:
def dct_2d(block):
    N = 8
    dct_block = np.zeros((N, N), dtype=float)
    for u in range(N):
        for v in range(N):
            sum_ = 0.0
            for x in range(N):
                for y in range(N):
                    sum_ += block[x, y] * np.cos((2 * x + 1) * u * np.pi / (2 * N)) * np.cos((2 * y + 1) * v * np.pi / (2 * N))
            c_u = 1 / np.sqrt(2) if u == 0 else 1
            c_v = 1 / np.sqrt(2) if v == 0 else 1
            dct_block[u, v] = 0.25 * c_u * c_v * sum_
    return dct_block

def apply_dct(component):
    height, width = component.shape
    dct_component = np.zeros_like(component, dtype=float)
    for i in range(0, height, 8):
        for j in range(0, width, 8):
            dct_component[i:i+8, j:j+8] = dct_2d(component[i:i+8, j:j+8])
    return dct_component

Quantization


In [ ]:
def quantize(block, q_matrix):
    quantized = np.zeros_like(block, dtype=int)
    for i in range(8):
        for j in range(8):
            quantized[i, j] = int(round(block[i, j] / q_matrix[i, j]))
    return quantized

def apply_quantization(dct_component, Q):
    quantized = np.zeros_like(dct_component)
    for i in range(0, dct_component.shape[0], 8):
        for j in range(0, dct_component.shape[1], 8):
            quantized[i:i+8, j:j+8] = quantize(dct_component[i:i+8, j:j+8], Q)
    return quantized

Zigzag scan for Run-Length Encoding


In [ ]:
def zigzag_order(block):
    zigzag = []
    indices = sorted(((i, j) for i in range(8) for j in range(8)), key=lambda x: (x[0] + x[1], -x[0] if (x[0] + x[1]) % 2 else x[1]))
    return [block[i, j] for i, j in indices]

def run_length_encode(block):
    zz = zigzag_order(block)
    rle = []
    zero_count = 0
    for value in zz:
        if value == 0:
            zero_count += 1
        else:
            rle.append((zero_count, value))
            zero_count = 0
    rle.append((0, 0))  # End of Block
    return rle

Full JPEG compression pipeline


In [ ]:
def jpeg_compress_pipeline(input_image_path, output_image_path):
    # Load image and convert to RGB array
    image = Image.open(input_image_path).convert('RGB')
    image_array = np.array(image)

    # Step 1: Convert to YCbCr
    ycbcr_array = rgb_to_ycbcr(image_array)

    # Step 2: Subsample chrominance channels
    y, cb, cr = subsample(ycbcr_array)

    # Step 3: Apply DCT
    y_dct = apply_dct(y)
    cb_dct = apply_dct(cb)
    cr_dct = apply_dct(cr)

    # Step 4: Define quantization matrices
    Q_Y = np.array([
        [16, 11, 10, 16, 24, 40, 51, 61],
        [12, 12, 14, 19, 26, 58, 60, 55],
        [14, 13, 16, 24, 40, 57, 69, 56],
        [14, 17, 22, 29, 51, 87, 80, 62],
        [18, 22, 37, 56, 68, 109, 103, 77],
        [24, 35, 55, 64, 81, 104, 113, 92],
        [49, 64, 78, 87, 103, 121, 120, 101],
        [72, 92, 95, 98, 112, 100, 103, 99]
    ])

    Q_C = np.array([
        [17, 18, 24, 47, 99, 99, 99, 99],
        [18, 21, 26, 66, 99, 99, 99, 99],
        [24, 26, 56, 99, 99, 99, 99, 99],
        [47, 66, 99, 99, 99, 99, 99, 99],
        [99, 99, 99, 99, 99, 99, 99, 99],
        [99, 99, 99, 99, 99, 99, 99, 99],
        [99, 99, 99, 99, 99, 99, 99, 99],
        [99, 99, 99, 99, 99, 99, 99, 99]
    ])

    # Step 5: Quantize
    y_quant = apply_quantization(y_dct, Q_Y)
    cb_quant = apply_quantization(cb_dct, Q_C)
    cr_quant = apply_quantization(cr_dct, Q_C)

    # Run-Length Encoding for each quantized block (not actually saving in JPEG format)
    y_rle = [run_length_encode(y_quant[i:i+8, j:j+8]) for i in range(0, y_quant.shape[0], 8) for j in range(0, y_quant.shape[1], 8)]
    cb_rle = [run_length_encode(cb_quant[i:i+8, j:j+8]) for i in range(0, cb_quant.shape[0], 8) for j in range(0, cb_quant.shape[1], 8)]
    cr_rle = [run_length_encode(cr_quant[i:i+8, j:j+8]) for i in range(0, cr_quant.shape[0], 8) for j in range(0, cr_quant.shape[1], 8)]

    # Note: JPEG encoding requires additional steps for saving to an actual JPEG file format.
    # We are simulating compression here, and Huffman encoding isn't implemented for brevity.

    print("JPEG compression pipeline completed")

    # Saving the original input image (since we don't create an actual JPEG file in this pipeline)
    image.save(output_image_path)
    print(f"Output image saved as {output_image_path}")

In [ ]:
jpeg_compress_pipeline("/content/new_sample.jpg", "compressed_output.jpg")